In [1]:
from sympy import *
import numpy as np

## Definiendo simbolos
t, r, theta, phi = symbols('t, r, \\theta, \\varphi')
dt, dr, dtheta, dphi = symbols('dt dr dtheta dphi')
l0, l1, l2, l3 = symbols("l_0 l_1 l_2 l_3")
l, p, q, n, L = symbols("lamb p q n L")

dAdt = "0"
dFdt = "0"
dGdt = "p/L"

ndmat = np.array([[[[[symbols("a") for _x in range(3)] for _y in range(3)] for _z in range(3)] for _w in range(3)] for _i in range(3)])

deldel = Function("DELDEL")
delf = Function("DELF")
box = Function("\\Box")

alfa = Function("a")(t, r, theta, phi)
F = Function("F")(t, r, theta, phi)
G = Function("G")(t, r, theta, phi)

def getFuncName(i):
    if i==0:
        return alfa.name
    if i==1:
        return F.name
    if i==2:
        return G.name

for _f in range(3):
    for _t in range(3):
        for _r in range(3):
            for _theta in range(3):
                for _phi in range(3):
                    ndmat[_f][_t][_r][_theta][_phi] = symbols(getFuncName(_f)+str(_t)+str(_r)+str(_theta)+str(_phi))



In [2]:
ndmat[0][2][1][1][1] 

a2111

In [3]:
def DELDEL(f, g):
    #canon = -f.diff(t)*g.diff(t) + 1/l1**2*f.diff(r)*g.diff(r) + 1/l1**2*f.diff(theta)*g.diff(theta) + 1/l2**2*f.diff(phi)*g.diff(phi)
    canon = -f.diff(t)*g.diff(t) + f.diff(r)*g.diff(r) + f.diff(theta)*g.diff(theta) + f.diff(phi)*g.diff(phi)
    return canon

def BOX(f):
    canon = -f.diff(t, 2) + f.diff(r, 2) + f.diff(theta, 2) + f.diff(phi, 2)
    #canon = -f.diff(t, 2) + 1/l1**2*f.diff(r, 2) + 1/l1**2*f.diff(theta, 2) + 1/l2 **2*f.diff(phi, 2)
    return canon

def DELf(f, g):
    canon = -(f*g.diff(t)).diff(t) + (f*g.diff(r)).diff(r)+ (f*g.diff(theta)).diff(theta) + (f*g.diff(phi)).diff(phi)
    #canon = -(f*g.diff(t)).diff(t) + 1/l1**2*(f*g.diff(r)).diff(r)+ 1/l1**2*(f*g.diff(theta)).diff(theta) + 1/l2**2*(f*g.diff(phi)).diff(phi)
    return canon


with evaluate(False):
    motion_alfa = (-box(alfa)+sin(alfa)*cos(alfa)*(deldel(F, F) + sin(F)**2*(deldel(G, G)))) + l*(
             sin(alfa)*cos(alfa)*(deldel(alfa, alfa)*(deldel(F, F)) - (deldel(alfa, F))**2) +
             sin(alfa)*cos(alfa)*sin(F)**2*(deldel(alfa, alfa)*(deldel(G, G)) - (deldel(alfa, G))**2) + 
             2*sin(alfa)**3*cos(alfa)*sin(F)**2*((deldel(F, F))*(deldel(G, G)) - (deldel(F, G))**2) -
             delf(sin(alfa)**2*(deldel(F, F)), alfa) + delf(sin(alfa)**2*(deldel(alfa, F)), F) -
             delf(sin(alfa)**2*sin(F)**2*(deldel(G, G)), alfa) + delf(sin(alfa)**2*sin(F)**2*(deldel(alfa, G)), G) )

    motion_F = -sin(alfa)**2*box(F)-2*sin(alfa)*cos(alfa)*(deldel(alfa, F))+sin(alfa)**2*sin(F)*cos(F)*(deldel(G, G)) + l*(
            sin(alfa)**2*sin(F)*cos(F)*(deldel(alfa, alfa)* deldel(G, G) - (deldel(alfa, G))**2) +
            sin(alfa)**4*sin(F)*cos(F)*((deldel(F, F))*(deldel(G, G)) - (deldel(F, G))**2) -
            delf(sin(alfa)**2*(deldel(alfa, alfa)), F) + delf(sin(alfa)**2*(deldel(alfa, F)), alfa) -
            delf(sin(alfa)**4*sin(F)**2*(deldel(G, G)), F) + delf(sin(alfa)**4*sin(F)**2*(deldel(F, G)), G) )

    motion_G = -sin(alfa)**2*sin(F)**2*box(G)-2*sin(alfa)*cos(alfa)*sin(F)**2*(deldel(alfa, G))-2*sin(alfa)**2*sin(F)*cos(F)*(deldel(F, G)) + l*(
            -delf(sin(alfa)**2*sin(F)**2*(deldel(alfa, alfa)), G) + delf(sin(alfa)**2*sin(F)**2*(deldel(alfa, G)), alfa) -
            delf(sin(alfa)**4*sin(F)**2*(deldel(F, F)), G) + delf(sin(alfa)**4*sin(F)**2*(deldel(F, G)), F) )

display(motion_alfa)
display(motion_F)
display(motion_G)

lamb*((((2*sin(a(t, r, \theta, \varphi))**3)*cos(a(t, r, \theta, \varphi)))*sin(F(t, r, \theta, \varphi))**2)*(DELDEL(F(t, r, \theta, \varphi), F(t, r, \theta, \varphi))*DELDEL(G(t, r, \theta, \varphi), G(t, r, \theta, \varphi)) - DELDEL(F(t, r, \theta, \varphi), G(t, r, \theta, \varphi))**2) + ((sin(a(t, r, \theta, \varphi))*cos(a(t, r, \theta, \varphi)))*sin(F(t, r, \theta, \varphi))**2)*(DELDEL(G(t, r, \theta, \varphi), G(t, r, \theta, \varphi))*DELDEL(a(t, r, \theta, \varphi), a(t, r, \theta, \varphi)) - DELDEL(a(t, r, \theta, \varphi), G(t, r, \theta, \varphi))**2) + (sin(a(t, r, \theta, \varphi))*cos(a(t, r, \theta, \varphi)))*(DELDEL(F(t, r, \theta, \varphi), F(t, r, \theta, \varphi))*DELDEL(a(t, r, \theta, \varphi), a(t, r, \theta, \varphi)) - DELDEL(a(t, r, \theta, \varphi), F(t, r, \theta, \varphi))**2) - DELF(DELDEL(F(t, r, \theta, \varphi), F(t, r, \theta, \varphi))*sin(a(t, r, \theta, \varphi))**2, a(t, r, \theta, \varphi)) + DELF(DELDEL(a(t, r, \theta, \varphi), F(t, r, \

lamb*(((sin(F(t, r, \theta, \varphi))*sin(a(t, r, \theta, \varphi))**2)*cos(F(t, r, \theta, \varphi)))*(DELDEL(G(t, r, \theta, \varphi), G(t, r, \theta, \varphi))*DELDEL(a(t, r, \theta, \varphi), a(t, r, \theta, \varphi)) - DELDEL(a(t, r, \theta, \varphi), G(t, r, \theta, \varphi))**2) + ((sin(F(t, r, \theta, \varphi))*sin(a(t, r, \theta, \varphi))**4)*cos(F(t, r, \theta, \varphi)))*(DELDEL(F(t, r, \theta, \varphi), F(t, r, \theta, \varphi))*DELDEL(G(t, r, \theta, \varphi), G(t, r, \theta, \varphi)) - DELDEL(F(t, r, \theta, \varphi), G(t, r, \theta, \varphi))**2) - DELF(DELDEL(a(t, r, \theta, \varphi), a(t, r, \theta, \varphi))*sin(a(t, r, \theta, \varphi))**2, F(t, r, \theta, \varphi)) + DELF(DELDEL(a(t, r, \theta, \varphi), F(t, r, \theta, \varphi))*sin(a(t, r, \theta, \varphi))**2, a(t, r, \theta, \varphi)) - DELF((sin(F(t, r, \theta, \varphi))**2*sin(a(t, r, \theta, \varphi))**4)*DELDEL(G(t, r, \theta, \varphi), G(t, r, \theta, \varphi)), F(t, r, \theta, \varphi)) + DELF((sin(F(t, 

lamb*(DELF((sin(F(t, r, \theta, \varphi))**2*sin(a(t, r, \theta, \varphi))**2)*DELDEL(a(t, r, \theta, \varphi), G(t, r, \theta, \varphi)), a(t, r, \theta, \varphi)) - DELF((sin(F(t, r, \theta, \varphi))**2*sin(a(t, r, \theta, \varphi))**2)*DELDEL(a(t, r, \theta, \varphi), a(t, r, \theta, \varphi)), G(t, r, \theta, \varphi)) - DELF((sin(F(t, r, \theta, \varphi))**2*sin(a(t, r, \theta, \varphi))**4)*DELDEL(F(t, r, \theta, \varphi), F(t, r, \theta, \varphi)), G(t, r, \theta, \varphi)) + DELF((sin(F(t, r, \theta, \varphi))**2*sin(a(t, r, \theta, \varphi))**4)*DELDEL(F(t, r, \theta, \varphi), G(t, r, \theta, \varphi)), F(t, r, \theta, \varphi))) - (sin(F(t, r, \theta, \varphi))*(2*sin(a(t, r, \theta, \varphi))**2))*cos(F(t, r, \theta, \varphi))*DELDEL(F(t, r, \theta, \varphi), G(t, r, \theta, \varphi)) - ((2*sin(a(t, r, \theta, \varphi)))*cos(a(t, r, \theta, \varphi)))*sin(F(t, r, \theta, \varphi))**2*DELDEL(a(t, r, \theta, \varphi), G(t, r, \theta, \varphi)) + (sin(F(t, r, \theta, \varphi)

In [4]:
def expandToDerivatives(func):
    p, q, s, b = map(Wild, 'pqsb')
    res = func.replace(deldel(p, q)*s, lambda p, q, s : DELDEL(p, q)*s)
    res = res.replace(delf(p, q)*s, lambda p, q, s : DELf(p, q)*s)
    return res.replace(box(p)*s, lambda p, s : BOX(p)*s)

In [5]:
res3a = (expandToDerivatives(motion_alfa))
display("Alfa done.")
res3f = (expandToDerivatives(motion_F))
display("F done.")
res3g = (expandToDerivatives(motion_G))
display("G done.")

'Alfa done.'

'F done.'

'G done.'

In [6]:
def getDelta(i):
    if i == t:
        return dt
    if r == i:
        return dr
    if theta == i:
        return dtheta
    if phi == i:
        return dphi

def getIndex(i):
    if t == i:
        return 0
    if r == i:
        return 1
    if theta == i:
        return 2
    if phi == i:
        return 3

def getFuncIndex(f):
    if f==F:
        return 1
    if f==G:
        return 2
    if f==alfa:
        return 0
    
def isThis(s, ix):
    if ix != None and s in ix.keys():
        return ix[s]
    else: 
        return 0
#http://www.iitg.ac.in/gtm/files/chap2.pdf
#https://www.uio.no/studier/emner/matnat/math/MAT-INF1100/h07/undervisningsmateriale/kap7.pdf
# 2: 1
# 1: 0
# 0: -1

def getDiscreteSymbol(ndmat, f, ix=None):
    canon = ndmat[getFuncIndex(f)][1+isThis(t, ix)][1+isThis(r, ix)][1+isThis(theta, ix)][1+isThis(phi, ix)]
    return canon
    
def FOD(f, i, wrt):
    if i==t:
        canon = (getDiscreteSymbol(ndmat, f, ix={i:0}) - getDiscreteSymbol(ndmat, f, ix={i:-1}))/(getDelta(i))
    else:
        canon = (getDiscreteSymbol(ndmat, f, ix={i:1}) - getDiscreteSymbol(ndmat, f, ix={i:-1}))/(2*getDelta(i))
    return canon

def SOD(f, i, j, wrt):
    if i==j:
        #if f!=wrt and i==t:
        #    canon = (getDiscreteSymbol(ndmat, f, ix={i:0}) - 2*getDiscreteSymbol(ndmat, f, ix={i:-1}) + getDiscreteSymbol(ndmat, f, ix={i:-2}))/(getDelta(i)**2)
        #else:
        canon = (getDiscreteSymbol(ndmat, f, ix={i:1}) - 2*getDiscreteSymbol(ndmat, f, ix={i:0}) + getDiscreteSymbol(ndmat, f, ix={i:-1}))/(getDelta(i)**2)
    else:
        if i != t and t != j:
            canon = (getDiscreteSymbol(ndmat, f, ix={i:1, j:1}) + getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:-1, j:1}) )/(4*getDelta(i)*getDelta(j))
        #backward difference
        elif (j==t):
            canon = (1/getDelta(i))*(((getDiscreteSymbol(ndmat, f, ix={i:1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:0, j:0}))/(getDelta(j))) - ((getDiscreteSymbol(ndmat, f, ix={i:1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:0, j:-1}))/(getDelta(j))))
            
            #canon = (1/(2*getDelta(i)*getDelta(j)))*(getDiscreteSymbol(ndmat, f, ix={i:0, j:1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:1})-getDiscreteSymbol(ndmat, f, ix={i:0, j:-1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}))
        elif (i == t):
            #canon = (1/(2*getDelta(i)*getDelta(j)))*(getDiscreteSymbol(ndmat, f, ix={i:1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:1, j:-1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}))
            canon = (1/getDelta(i))*(((getDiscreteSymbol(ndmat, f, ix={i:0, j:1})-getDiscreteSymbol(ndmat, f, ix={i:0, j:0}))/(getDelta(j))) - ((getDiscreteSymbol(ndmat, f, ix={i:-1, j:1}) - getDiscreteSymbol(ndmat, f, ix={i:-1, j:0}))/(getDelta(j))))
            
        else:
            print("what")
    return canon

def discreti(arg, wrt):
    if len(arg.variables) == 1:
        return FOD(arg.expr, arg.variables[0], wrt)
    elif len(arg.variables) == 2:
        return SOD(arg.expr, arg.variables[0], arg.variables[1], wrt)
    else:
        return None

In [7]:
def discretizeDerivatives(func, wrt):
    res = func.replace(lambda arg: arg.is_Derivative, lambda arg: discreti(arg, wrt))
    res = res.replace(F, getDiscreteSymbol(ndmat, F))
    res = res.replace(G, getDiscreteSymbol(ndmat, G))
    return res.replace(alfa, getDiscreteSymbol(ndmat, alfa))

In [ ]:
res7a = discretizeDerivatives(res3a, alfa)

res7f = discretizeDerivatives(res3f, F)

res7g = discretizeDerivatives(res3g, G)

In [ ]:
expra = ccode(res7a)
exprf = ccode(res7f)
exprg = ccode(res7g)

In [ ]:
def getSymbolsUsed(expr):
    syms = ""
    for _y in expr.free_symbols:
        syms += str(_y)+" "
    return syms[:-1]

In [ ]:
symma = getSymbolsUsed(res7a)
print("Symbols for a: \n [", symma, "]")

symmf = getSymbolsUsed(res7f)
print("Symbols for F: \n [", symmf, "]")

symmg = getSymbolsUsed(res7g)
print("Symbols for G: \n [", symmg, "]")


In [ ]:
import sage as sg
from sage.all import *
var(symma)

def solveWithSage(expr, symbols, x):
    wata = eval(expr)
    resa = sg.symbolic.relation.solve(wata, x)
    return resa[0]

In [ ]:
def getDerivative(eq, var):
    if type(eq) == int:
        return 0
    else:
        try:
            return eq.derivative(var)
        except:
            return 0

def solveForFirstState(expr, dfdt, func, hasMixed):
    auxt = eval(func+"1111") - dfdt*2*dt
    
    
    firsta = expr.subs({eval(func+"0111") : auxt})
    if hasMixed:
        auxr = eval(func+"1211")-eval(func+"1111")+eval(func+"0111")-dt*dr*getDerivative(dfdt, r)
        auxtheta = eval(func+"1121")-eval(func+"1111")+eval(func+"0111")+dt*dr*getDerivative(dfdt, theta)
        auxphi = eval(func+"1112")-eval(func+"1111")+eval(func+"0111")+dt*dr*getDerivative(dfdt, phi)
        firsta = firsta.subs({eval(func+"0211") : auxr})
        firsta = firsta.subs({eval(func+"0121") : auxtheta})
        firsta = firsta.subs({eval(func+"0112") : auxphi})
        firsta = firsta.subs({eval(func+"0111") : auxt})
    firstaa = sg.symbolic.relation.solve(firsta, eval(func+"2111"))
    try:
        return firstaa[0]#.collect_common_factors().full_simplify().collect_common_factors()
    except:
        return firstaa

def replaceOtherFunctionsFirstState(expr, func1, func2, df1dt, df2dt):
    auxtf1 = eval(func1+"1111") - df1dt*dt
    auxtf2 = eval(func2+"1111") - df2dt*dt
    
    firsta = expr.subs({eval(func1+"2111") : auxtf1})
    firsta = firsta.subs({eval(func2+"2111") : auxtf2})
    return firsta

In [ ]:
ra = solveWithSage(expra, symma, a2111)
print(ra)

In [ ]:
fa = solveForFirstState(ra, eval(dAdt), "a", True)
fa = replaceOtherFunctionsFirstState(fa, "F", "G", eval(dFdt), eval(dGdt))
fa

In [ ]:
rf = solveWithSage(exprf, symmf, F2111)
print(rf)

In [ ]:
ff = solveForFirstState(rf, eval(dFdt), "F", True)
ff = replaceOtherFunctionsFirstState(ff, "a", "G", eval(dAdt), eval(dGdt))

ff

In [ ]:
rg = solveWithSage(exprg, symmg, G2111)
print(rg)

In [ ]:
fg = solveForFirstState(rg, eval(dAdt), "G", True)
fg = replaceOtherFunctionsFirstState(fg, "a", "F", eval(dAdt), eval(dFdt))

fg

In [ ]:
import re
def i2i(arg, index):
    if index == "0":
        return "("+arg+"-1)"
    if index == "1":
        return "("+arg+")"
    if index == "2":
        return "("+arg+"+1)"
    #if index == "3":
    #    return "("+arg+"+1)"
def i2iTime(arg, index):
    if index == "0":
        return "(tm1)"
    if index == "1":
        return "(t)"
    if index == "2":
        return "(tp1)"
    #if index == "3":
    #    return "(tp1)"    
def getCArray(ids):
    res = ""
    args = ["t", "r", "theta", "phi"]
    sizes = ["L", "M", "N", "O"]
    for i, c in enumerate(ids):
        if args[i] == "t":
            res += i2iTime(args[i], c)
        else:
            res += i2i(args[i], c)
        for j,s in enumerate(sizes[i+1:]):
            res+="*"+s
        res+="+"
    return res[:-1]


def getExpressionInC(expr):
    
    traduc = ccode(expr)

    indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d\d\d', traduc)]
    for i in reversed(indexes):
        traduc = traduc[:i+1]+"["+ getCArray(traduc[i+1:i+5]) +"]"+ traduc[i+5:]
    return traduc

In [ ]:
expr_a = getExpressionInC(ra._sympy_())
expr_fa = getExpressionInC(fa._sympy_())
expr_fa

In [ ]:
expr_f = getExpressionInC(rf._sympy_())
expr_ff = getExpressionInC(ff._sympy_())
expr_f

In [ ]:
expr_g = getExpressionInC(rf._sympy_())
expr_fg = getExpressionInC(fg._sympy_())
expr_g[41:]

In [ ]:
def replaceInFile(filename, match, new):
    #read input file
    fin = open(filename, "rt")
    #read file contents to string
    data = fin.read()
    #replace all occurrences of the required string
    data = data.replace(match, new)
    #close the input file
    fin.close()
    #open the input file in write mode
    fin = open(filename, "wt")
    #overrite the input file with the resulting data
    fin.write(data)
    #close the file
    fin.close()

In [27]:
#replaceInFile("../cpu/src/EquationAlfa.cpp", "---a---", expr_a[41:])
replaceInFile("../cpu/src/EquationAlfa.cpp", "---fa---", expr_fa[41:])
#replaceInFile("../cpu/src/EquationF.cpp", "---F---", expr_f[41:])
replaceInFile("../cpu/src/EquationF.cpp", "---fF---", expr_ff[41:])
#replaceInFile("../cpu/src/EquationG.cpp", "---G---", expr_g[41:])
replaceInFile("../cpu/src/EquationG.cpp", "---fG---", expr_fg[41:])


In [13]:
import pickle

with open('F[t+1].pickle', 'wb') as f:
    pickle.dump(traduc, f)

In [6]:
import pickle

with open('F[t+1].pickle') as f:
    traduc = pickle.load(f)

NameError: name 'traduc' is not defined